In [3]:
import os
import requests
import base64
from datetime import datetime, timedelta
from urllib.parse import urlencode

In [4]:
auth_url = "https://accounts.spotify.com/api/token"
client_id = os.getenv("SPOTIPY_CLIENT_ID")
client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")
redirect_uri = os.getenv("SPOTIPY_REDIRECT_URI")
base_url = "https://api.spotify.com/v1/" # base URL of all Spotify API endpoints

In [63]:
class SpotifyAPI():
    access_token = None
    access_token_expiry = datetime.now()
    client_id = None
    client_secret = None
    auth_url = 'https://accounts.spotify.com/api/token'
    
    
    def __init__(self, client_id, client_secret):
        '''
        Initialises the Spotify client with the client ID and secret
        '''
        self.client_id = client_id
        self.client_secret = client_secret
    
    
    def get_token_header(self):
        '''
        Returns the token header in base64 encoding necessary for the authorisation post request
        '''
        client_credentials_b64 = self.get_client_credentials()
        return {"Authorization": "Basic {credentials}".format(credentials=client_credentials_b64)}
    
    
    def get_token_data(self):
        '''
        Returns the token data necessary for the authorisation post request
        '''
        return {"grant_type": "client_credentials"}
    
    
    def get_client_credentials(self):
        ''' 
        Returns a base 64 encoded authorisation string
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id is None or client_secret is None:
            raise Exception("You must set a client ID and a client secret")
        client_credentials = f'{client_id}:{client_secret}'
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        return client_credentials_b64.decode()
    
    
    def authorise(self):
        '''
        Authorises the client, setting the authorisation token and returning True if successful
        '''
        auth_url = self.auth_url
        auth_data = self.get_token_data()
        auth_headers = self.get_token_header()
        auth_response = requests.post(auth_url, data=auth_data, headers=auth_headers)
        if auth_response.status_code not in range(200,299):
            raise Exception('Could not authenticate client')
            return False
        auth_response_data = auth_response.json()
        self.access_token = auth_response_data["access_token"]
        now = datetime.now()
        expires_in = auth_response_data['expires_in']
        expires = now + timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_expired = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.now()
        if expires < now:
            self.authorise()
            return self.access_token
        elif token is None:
            self.authorise()
            return self.access_token 
        return token
            
        
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {'Authorization': f'Bearer {access_token}'}
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums'):
        endpoint = f'{base_url}{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
        
        
    def get_album(self, _id):
        return self.get_resource(_id, 'albums')
    
    
    def get_artist(self, _id):
        return self.get_resource(_id, 'artists')
    
    
    def search(self, query, search_type='track'):
        endpoint = f'{base_url}search'
        headers = self.get_resource_header()
        data = urlencode({'q':query,'type':search_type.lower()})
        url = f'{endpoint}?{data}'
        r = requests.get(url, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
        

In [64]:
spotify = SpotifyAPI(client_id, client_secret)

In [65]:
spotify.authorise()

True

In [66]:
spotify.search('A lannister always plays his debts', 'track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=A+lannister+always+plays+his+debts&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
       'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
       'id': '1hCkSJcXREhrodeIHQdav8',
       'name': 'Ramin Djawadi',
       'type': 'artist',
       'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
   

In [67]:
spotify.get_artist('1hCkSJcXREhrodeIHQdav8')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
 'followers': {'href': None, 'total': 502240},
 'genres': ['german soundtrack',
  'scorecore',
  'soundtrack',
  'video game music'],
 'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
 'id': '1hCkSJcXREhrodeIHQdav8',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb91e7dd670bc7d3ec7d1cbc6f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab6761610000517491e7dd670bc7d3ec7d1cbc6f',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f17891e7dd670bc7d3ec7d1cbc6f',
   'width': 160}],
 'name': 'Ramin Djawadi',
 'popularity': 71,
 'type': 'artist',
 'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}